### Download and prepare KDD1999 10% dataset

In [1]:
import os
import time
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
import zipfile
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row

def load_kddcup_data(filepath):
    columns = [
        'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
        'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
        'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
        'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
        'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
        'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
        'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
        'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
        'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label'
    ]
    
    data = pd.read_csv(filepath, header=None, names=columns)
    return data

def preprocess_kddcup_data(data):
    # Separate features and labels
    X = data.drop(columns=['label'])
    
    # Define categorical and numerical columns
    categorical_features = ['protocol_type', 'service', 'flag']
    numerical_features = [col for col in X.columns if col not in categorical_features]
    
    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            # Convert all numerical features to standard normal destribution
            ('num', StandardScaler(), numerical_features),
            # One hot encode categorical features
            # ('cat', OneHotEncoder(), categorical_features)
        ]
    )
    
    return preprocessor.fit_transform(X)

def download_kddcup99(url, destination_file_name, destination_folder="/home/jovyan/work/data"):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        
    compressed_file = os.path.join(destination_folder, destination_file_name + ".gz")
    extracted_file = os.path.join(destination_folder, destination_file_name)
    
    # Download the dataset if not already downloaded
    if not os.path.exists(compressed_file):
        print("Downloading dataset...")
        urlretrieve(url, compressed_file)
        print("Download complete.")
    
    # Extract the dataset if not already extracted
    if not os.path.exists(extracted_file):
        print("Extracting dataset...")
        os.system(f"gunzip {compressed_file}")
        print("Extraction complete.")
    
    return extracted_file

def download_kddcup99(url, destination_file_name, destination_folder="/home/jovyan/work/data"):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        
    compressed_file = os.path.join(destination_folder, destination_file_name + ".gz")
    extracted_file = os.path.join(destination_folder, destination_file_name)
    
    # Download the dataset if not already downloaded
    if not os.path.exists(compressed_file):
        print("Downloading dataset...")
        urlretrieve(url, compressed_file)
        print("Download complete.")
    
    # Extract the dataset if not already extracted
    if not os.path.exists(extracted_file):
        print("Extracting dataset...")
        os.system(f"gunzip {compressed_file}")
        print("Extraction complete.")
    
    return extracted_file

In [2]:
url = "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz"
data_file_path = download_kddcup99(url, "kdd_10_percent")

In [3]:
k = 15  # Number of clusters
max_iterations = 5
num_partitions = 10

In [4]:
data = load_kddcup_data(data_file_path)
processed_data = preprocess_kddcup_data(data)

In [5]:
from sklearn.datasets import make_blobs
processed_data, y = make_blobs(n_samples=200000, n_features=25, centers=k, random_state=42)

### Simple KMeans

In [6]:
import time
import random
import math

def initialize_centroids(X, k, seed=314):
    """
    Randomly initialize centroids from the dataset with a fixed seed for reproducibility.
    """
    random.seed(seed)
    random_indices = random.sample(range(len(X)), k)
    centroids = [X[idx][:] for idx in random_indices]  # Copy selected points
    return centroids

def euclidean_distance(point1, point2):
    """Compute the Euclidean distance between two points."""
    return math.sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(point1, point2)))

def compute_distances(X, centroids):
    """
    Compute the distance between each data point and each centroid.
    """
    distances = []
    for point in X:
        distances.append([euclidean_distance(point, centroid) for centroid in centroids])
    return distances

def assign_clusters(distances):
    """
    Assign each data point to the closest centroid.
    """
    labels = []
    for distance_list in distances:
        min_distance_index = distance_list.index(min(distance_list))
        labels.append(min_distance_index)
    return labels

def update_centroids(X, labels, k):
    """
    Update centroids as the mean of all points assigned to each cluster.
    """
    centroids = [[0] * len(X[0]) for _ in range(k)]
    counts = [0] * k
    
    # Sum points for each cluster
    for idx, label in enumerate(labels):
        for dim in range(len(X[idx])):
            centroids[label][dim] += X[idx][dim]
        counts[label] += 1
    
    # Divide by the count to compute the mean
    for i in range(k):
        if counts[i] > 0:  # Avoid division by zero
            centroids[i] = [val / counts[i] for val in centroids[i]]
    return centroids

def has_converged(new_centroids, old_centroids, tolerance):
    """
    Check if centroids have converged based on a given tolerance.
    """
    for nc, oc in zip(new_centroids, old_centroids):
        if any(abs(n - o) > tolerance for n, o in zip(nc, oc)):
            return False
    return True

def simple_kmeans(X, k, max_iters=150, tolerance=0):
    """
    Perform the K-Means clustering algorithm.
    """
    start_time = time.time()
    centroids = initialize_centroids(X, k)
    
    for i in range(max_iters):
        iteration_time = time.time()
        old_centroids = [c[:] for c in centroids]  # Deep copy
        distances = compute_distances(X, centroids)
        labels = assign_clusters(distances)
        centroids = update_centroids(X, labels, k)
        
        print(f"Iteration: {i}\ttime taken: {time.time() - iteration_time:.4f} seconds")
        
        # Check for convergence
        if has_converged(centroids, old_centroids, tolerance):
            print(f"K-Means converged after {i+1} iterations.")
            break
    
    print(f"Total time: {time.time() - start_time:.4f} seconds")
    return centroids, labels


In [7]:
centroids, labels = simple_kmeans(processed_data, k, max_iters=max_iterations)

Iteration: 0	time taken: 14.9475 seconds
Iteration: 1	time taken: 13.6388 seconds
Iteration: 2	time taken: 13.5315 seconds
Iteration: 3	time taken: 13.6660 seconds
Iteration: 4	time taken: 13.6080 seconds
Iteration: 5	time taken: 13.5686 seconds
Iteration: 6	time taken: 13.6324 seconds
Iteration: 7	time taken: 13.5577 seconds
Iteration: 8	time taken: 13.5366 seconds
Iteration: 9	time taken: 13.5449 seconds
Iteration: 10	time taken: 13.6500 seconds
Iteration: 11	time taken: 13.5445 seconds
Iteration: 12	time taken: 13.7197 seconds
Iteration: 13	time taken: 13.9401 seconds
Iteration: 14	time taken: 13.5913 seconds
Iteration: 15	time taken: 13.5854 seconds
Iteration: 16	time taken: 13.5893 seconds
Iteration: 17	time taken: 13.5277 seconds
Iteration: 18	time taken: 13.6221 seconds
Iteration: 19	time taken: 13.5762 seconds
Total time: 273.5813 seconds


In [8]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row

In [9]:
# Create Spark session
spark = SparkSession.builder \
    .appName("DockerClusterApp") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/19 13:19:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
def kmeans_to_spark_df(X, labels):
    """
    Convert NumPy array and cluster labels into a PySpark DataFrame with features and predictions.
    """
    # Convert NumPy data to a list of Rows with features and predictions
    rows = [Row(features=Vectors.dense(X[i]), prediction=int(labels[i])) for i in range(len(labels))]
    return spark.createDataFrame(rows)

In [11]:
spark_df = kmeans_to_spark_df(processed_data, labels)

evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="prediction", metricName="silhouette")
silhouette_score = evaluator.evaluate(spark_df)

print(f"Silhouette Score: {silhouette_score}")

spark.stop()

25/01/19 13:19:54 WARN TaskSetManager: Stage 0 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.
25/01/19 13:19:56 WARN TaskSetManager: Stage 1 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.
25/01/19 13:20:00 WARN TaskSetManager: Stage 3 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.


Silhouette Score: 0.680941640132225


# PKMEANS

In [12]:
def assign_clusters_to_data(X, centroids):
    labels = []
    for point in X:
        distances = np.linalg.norm(centroids - point, axis=1)
        labels.append(np.argmin(distances))
    return np.array(labels)

In [13]:
import random 
import time
import math
import logging
import itertools
import typing
from numpy import array
import numpy as np
import pandas as pd
from scipy import spatial
import matplotlib.pyplot as plt
from pprint import pprint
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.mllib.clustering import KMeans, KMeansModel
from itertools import groupby, compress

from typing import Tuple, Sequence

def euclidean_dist(v1, v2):
    return np.sqrt(np.sum((v1 - v2) ** 2, axis=1))

def closest_centroid(points, centroids):
    distances = np.sqrt(((points[:, None] - centroids[None, :]) ** 2).sum(axis=2))
    closest_centroids_indices = np.argmin(distances, axis=1)
    return closest_centroids_indices

def calc_partition_centroid_means(partition, centroids):
    points = np.array(list(partition)) 
    if len(points) == 0:  # Handle empty partitions
        return []

    closest_indices = closest_centroid(points, centroids.value)
    
    # Combine points with their respective closest centroids
    data = pd.DataFrame({
        "Centroid": closest_indices,
        "Point": list(points)
    })

    # Expand Point into multiple dimensions
    point_df = pd.DataFrame(data['Point'].tolist(), index=data.index)
    combined_df = pd.concat([data['Centroid'], point_df], axis=1)

    # Group by Centroid and calculate sum and count for each group
    grouped = combined_df.groupby('Centroid').agg(['sum', 'count'])

    grouped.columns = ['_'.join(map(str, col)).strip() for col in grouped.columns.to_flat_index()]
    grouped.reset_index(inplace=True)

    # Extract sum and count separately
    point_sums = grouped[[f'{i}_sum' for i in range(points.shape[1])]].to_numpy()
    counts = grouped['0_count'].to_numpy()

    # Return 2 array with each row: [Centroid, *points sums, count]
    return np.column_stack([grouped['Centroid'].to_numpy(), point_sums, counts])

def aggregate_means(rdd):
    partition_means = np.concatenate(rdd.collect(), axis=0)

    # Extract dimensions and count
    num_dimensions = int(partition_means.shape[1] - 2)  # Exclude centroid and count
    sums = partition_means[:, 1:1 + num_dimensions]
    counts = partition_means[:, -1]

    # Aggregate sums and counts by centroid
    data = pd.DataFrame(partition_means, columns=['Centroid'] + [f'dim_{i}_sum' for i in range(num_dimensions)] + ['count'])
    aggregated = data.groupby('Centroid').sum()
    aggregated_sums = aggregated[[f'dim_{i}_sum' for i in range(num_dimensions)]].to_numpy()
    aggregated_counts = aggregated['count'].to_numpy()

    # Compute weighted averages
    new_centroids = aggregated_sums / aggregated_counts[:, None]

    return pd.DataFrame(new_centroids, index=aggregated.index)

def handle_missing_centroids(aggregated_centroids, old_centroids):
    num_centroids = len(old_centroids)
    missing_centroids = set(range(num_centroids)) - set(aggregated_centroids.index)
    for ix in missing_centroids:
        # Copy old centroids to replace the missing
        aggregated_centroids.loc[ix] = old_centroids[ix]
    return aggregated_centroids.sort_index().to_numpy()

def closest_centroids(data, centroids):
    return data.mapPartitions(lambda partition: [calc_partition_centroid_means(partition, centroids)])

def handle_missing_centorids(aggregated_centroids, old_centroids):
    num_centorids = len(old_centroids)
    missing_centroids = set(range(num_centorids)) - set(aggregated_centroids.index)
    for ix in missing_centroids:
        # Copy old centroids to replace the missing
        print("handled missing")
        aggregated_centroids.loc[ix] = old_centroids[ix]
    return aggregated_centroids.sort_index().to_numpy()

def calc_error(new_centroids, old_centroids):
    return np.sum(euclidean_dist(new_centroids, old_centroids))

def pkmeans(data, n, max_iterations=150, stop_distance=0.001):
    print(time.asctime(), "Started")
    start_time = time.time()
    init_centroids = np.array(data.takeSample(False, n, seed=42)) 
    centroids = sc.broadcast(init_centroids)

    iteration = 1
    error = float("inf")
    while error > stop_distance and iteration <= max_iterations:
        loop_start = time.time()
        closest_centroids_rdd = closest_centroids(data, centroids)
        aggregated_centroids = aggregate_means(closest_centroids_rdd)
        new_centroids = handle_missing_centorids(aggregated_centroids, centroids.value)
        error = calc_error(new_centroids, centroids.value)
        print("{3} Iteration #{0}\tDistance between old and new centroids: {1:.4f}\tIteration took: {2:.4f} sec".format(
            iteration, error, time.time() - loop_start, time.asctime()))
        
        # Update centroids
        centroids = sc.broadcast(new_centroids) 
        iteration += 1

    print(f"Total time: {time.time() - start_time}")

    return centroids.value
    


In [14]:
np.savetxt("data.csv", processed_data, delimiter=",")

In [15]:
spark = SparkSession.builder \
    .appName("PKmeans") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

sc = spark.sparkContext

rdd = sc.textFile("data.csv")

parsed_data = rdd.map(lambda line: [float(x) for x in line.split(",")])
parsed_data = parsed_data.cache()

In [16]:
final_centorids = pkmeans(parsed_data, k, max_iterations=max_iterations)

Sun Jan 19 13:20:07 2025 Started


Sun Jan 19 13:20:11 2025 Iteration #1	Distance between old and new centroids: 165.1339	Iteration took: 0.9237 sec


handled missing
Sun Jan 19 13:20:12 2025 Iteration #2	Distance between old and new centroids: 55.2437	Iteration took: 0.9337 sec


handled missing
Sun Jan 19 13:20:13 2025 Iteration #3	Distance between old and new centroids: 0.8852	Iteration took: 0.8377 sec


handled missing
Sun Jan 19 13:20:14 2025 Iteration #4	Distance between old and new centroids: 0.5326	Iteration took: 0.8267 sec


...

handled missing
Sun Jan 19 13:20:24 2025 Iteration #16	Distance between old and new centroids: 0.0964	Iteration took: 0.8158 sec


handled missing
Sun Jan 19 13:20:25 2025 Iteration #17	Distance between old and new centroids: 0.0837	Iteration took: 0.7919 sec


handled missing
Sun Jan 19 13:20:26 2025 Iteration #18	Distance between old and new centroids: 0.0734	Iteration took: 0.8220 sec


handled missing
Sun Jan 19 13:20:26 2025 Iteration #19	Distance between old and new centroids: 0.0794	Iteration took: 0.8717 sec


handled missing
Sun Jan 19 13:20:27 2025 Iteration #20	Distance between old and new centroids: 0.0782	Iteration took: 0.8230 sec
Total time: 20.465823650360107


In [17]:
labels = assign_clusters_to_data(processed_data, final_centorids)
spark_df = kmeans_to_spark_df(processed_data, labels)
evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="prediction", metricName="silhouette")
silhouette_score = evaluator.evaluate(spark_df)

print(f"Silhouette Score: {silhouette_score}")

25/01/19 13:20:32 WARN TaskSetManager: Stage 22 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.
25/01/19 13:20:33 WARN TaskSetManager: Stage 23 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.
25/01/19 13:20:37 WARN TaskSetManager: Stage 25 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.


Silhouette Score: 0.635128597168241


In [18]:
import os
import random
import time
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
import numpy as np

def calculate_similarity(group1, group2):
    """
    Calculate the similarity score between two centroid groups.
    The similarity is the inverse of the sum of distances between corresponding centroids.
    """
    distances = [np.linalg.norm(c1 - c2) for c1, c2 in zip(group1, group2)]
    return 1 / sum(distances) if sum(distances) != 0 else float('inf')


def find_most_dissimilar_group(target_group, groups):
    """
    Find the most dissimilar group to the target group.
    """
    max_dissimilarity = -float('inf')
    most_dissimilar = None
    
    for group in groups:
        if np.array_equal(target_group, group):
            continue  # Skip self-comparison
        
        similarity = calculate_similarity(target_group, group)
        if similarity > max_dissimilarity:
            max_dissimilarity = similarity
            most_dissimilar = group
    
    return most_dissimilar


def adgp(groups):
    """
    Generate new centroid groups using the Average of Dissimilar Group Pairs (ADGP).
    """
    new_groups = []
    group_count = len(groups)
    
    for i, group1 in enumerate(groups):
        group2 = find_most_dissimilar_group(group1, groups)
        
        # Compute the average of corresponding centroids to form a new group
        new_group = [(c1 + c2) / 2 for c1, c2 in zip(group1, group2)]
        new_groups.append(new_group)
    
    return new_groups

# Permutation function to align centroids across groups
def permute_centroids(centroid_groups):
    base_group = centroid_groups[0]
    permuted_groups = []

    for group in centroid_groups[1:]:
        # Track matched indices to prevent duplication
        matched = set()
        permuted_group = []
        for base_c in base_group:
            # Find the closest unmatched centroid
            distances = [(i, np.linalg.norm(base_c - c)) for i, c in enumerate(group) if i not in matched]
            if distances:
                closest_idx = min(distances, key=lambda x: x[1])[0]
                permuted_group.append(group[closest_idx])
                matched.add(closest_idx)
            else:
                # Handle unmatched cases by using a default
                permuted_group.append(base_c)
        permuted_groups.append(permuted_group)
    return [base_group] + permuted_groups

def initialize_centroid_groups(parsed_data, k, s):
    # Collect a small subset of the data for initialization
    sample_data = parsed_data.takeSample(False, k * s, seed=1)
    centroid_groups = [
        sample_data[i * k:(i + 1) * k] for i in range(s)
    ]
    return np.array(centroid_groups)

def mux_kmeans(data, k, s, max_iterations=20):
    start_time = time.time()
    
    # Initialize centroid groups
    initial_centroids_groups = initialize_centroid_groups(data, k, s)
    
    # Mux-Kmeans main loop
    for iteration in range(max_iterations):
        iteration_time = time.time()
        twcv_scores = []
        
        # Evaluate centroid groups
        for centroids in initial_centroids_groups:
            centroids_broadcast = sc.broadcast(centroids)
    
            # Assign points to clusters
            clustered_rdd = data.map(
                lambda p: (
                    np.argmin([np.linalg.norm(np.subtract(p, c)) for c in centroids_broadcast.value]),
                    (p, 1)
                )
            ) # (Cluster index, (point, 1))

            # Recalculate centroids
            new_centroids = (
                clustered_rdd
                .reduceByKey(lambda x, y: (np.add(x[0], y[0]), x[1] + y[1]))  # Sum points and count
                .map(lambda x: (x[0], np.array(x[1][0]) / x[1][1]))  # Compute new centroids
                .collectAsMap()
            )

            new_centroids_arr = np.array(
                    [new_centroids[j] if j in new_centroids else centroids[j] for j in range(len(centroids))]
            )
    
            # Update the centroids to the new centroids
            centroids_broadcast = sc.broadcast(new_centroids_arr)
            
            # Assign points to the nearest new centroids
            centroid_to_point = data.map(
                lambda point: (
                    np.argmin([np.linalg.norm(point - c) for c in centroids_broadcast.value]), # cluster_id
                    point # original point
                )
            )
    
            # Calculate TWCV for the current group
            # Row is (cluster_id, point)
            twcv = centroid_to_point.map(lambda row: np.linalg.norm(row[1] - centroids_broadcast.value[row[0]]) ** 2).sum()
            twcv_scores.append((new_centroids_arr, twcv))
    
        # Prune and incubate
        twcv_scores.sort(key=lambda x: x[1])
        best_groups = twcv_scores[:s // 2]
        best_centroids = [x[0] for x in best_groups]
    
        # Permutation
        aligned_centroid_groups = permute_centroids(best_centroids)
        
        # Incubate new groups
        new_centroid_groups = []
        for group in aligned_centroid_groups:
            new_centroid_groups.append(adgp(group))
    
        # Prepare for the next iteration
        initial_centroids_groups = best_centroids + new_centroid_groups
        
        best_twcv = min(twcv_scores, key=lambda x: x[1])[1]

        print(f"Iteration: {iteration}\ttime taken: {time.time() - iteration_time:.4f} seconds\tBest TWCV: {best_twcv}")

    # Select the final best group
    final_group = min(twcv_scores, key=lambda x: x[1])
    print(f"Total time: {time.time() - start_time:.4f} seconds")
    return final_group[0]

In [19]:
spark = SparkSession.builder \
    .appName("Mux-KMeans") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

sc = spark.sparkContext

rdd = sc.textFile("data.csv")

parsed_data = rdd.map(lambda line: [float(x) for x in line.split(",")])
parsed_data = parsed_data.cache()
final_centorids = mux_kmeans(parsed_data, k, s=6)
spark.stop()


25/01/19 13:20:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Iteration: 0	time taken: 30.9296 seconds	Best TWCV: 30782508.941039987


Iteration: 1	time taken: 32.2628 seconds	Best TWCV: 23770604.203099255


Iteration: 2	time taken: 32.2918 seconds	Best TWCV: 19155204.29884699


Iteration: 3	time taken: 31.9060 seconds	Best TWCV: 19154543.21682926


...

Iteration: 15	time taken: 32.0326 seconds	Best TWCV: 19153592.75889112


Iteration: 16	time taken: 31.8034 seconds	Best TWCV: 19153576.651242007


Iteration: 17	time taken: 31.6155 seconds	Best TWCV: 19153562.616959266


Iteration: 18	time taken: 3138.1754 seconds	Best TWCV: 19153549.122602116


Iteration: 19	time taken: 55.4126 seconds	Best TWCV: 19153538.682184897
Total time: 3771.2822 seconds


In [20]:
spark = SparkSession.builder \
    .appName("Mux-KMeans-Silhouette") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

labels = assign_clusters_to_data(processed_data, final_centorids)
spark_df = kmeans_to_spark_df(processed_data, labels)
evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="prediction", metricName="silhouette")
silhouette_score = evaluator.evaluate(spark_df)

print(f"Silhouette Score: {silhouette_score}")
spark.stop()

25/01/19 14:23:36 WARN TaskSetManager: Stage 0 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.
25/01/19 14:23:37 WARN TaskSetManager: Stage 1 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.
25/01/19 14:23:41 WARN TaskSetManager: Stage 3 contains a task of very large size (5768 KiB). The maximum recommended task size is 1000 KiB.


Silhouette Score: 0.732183963349354


In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output

dropdown_algorithm = widgets.Dropdown(
    options=['KMeans', 'KMeans', 'Mux-Kmeans'],
    value='KMeans',
    description='Select Algorithm:',
    style = {'description_width':'initial'},
    layout = {'width':'300px'}
)

dropdown_dataset = widgets.Dropdown(
    options=['KDD1999-10%', 'Syntetic200K-5', 'Syntetic200K-15'],
    value='Syntetic200K-5',
    description='Select Dataset:',
    style = {'description_width':'initial'},
    layout = {'width':'300px'}
)

# Sliders
k_slider = widgets.IntSlider(value=5, min=5, max=15, step=1, description='K:', style = {'description_width':'initial'},
    layout = {'width':'300px'})
max_iterations_slider = widgets.IntSlider(value=5, min=5, max=20, step=1, description='Max Iterations:',style = {'description_width':'initial'},
    layout = {'width':'300px'})
s_slider = widgets.IntSlider(value=6, min=4, max=10, step=1, description='S:', style = {'description_width':'initial'},
    layout = {'width':'300px'})

# Container to display sliders
slider_container = widgets.VBox([k_slider, max_iterations_slider])

# Output area for results
output_area = widgets.Output()

# Update function to handle conditional slider
def update_sliders(change):
    if change['new'] == 'Mux-Kmeans':
        slider_container.children = [k_slider, max_iterations_slider, s_slider]
    else:
        slider_container.children = [k_slider, max_iterations_slider]

# Function to execute and display results
def execute_action(b):
    with output_area:
        clear_output()
        result = {
            'Selected Option': dropdown.value,
            'Slider 1 Value': slider1.value,
            'Slider 2 Value': slider2.value,
            'Slider 3 Value': slider3.value,
        }
        if dropdown.value == 'Option 3':
            result['Conditional Slider Value'] = slider4.value
        print("Results:", result)

# Observe dropdown changes
dropdown_algorithm.observe(update_sliders, names='value')

# Execute button
execute_button = widgets.Button(description='Execute')
execute_button.on_click(execute_action)


In [4]:
display(widgets.VBox([dropdown_algorithm, dropdown_dataset, slider_container, execute_button, output_area]))